In [1]:
%cd /scratch/edk202/ngram-prep

%env LC_ALL=C.UTF-8
%env LANG=C.UTF-8

%pip install -e . --no-build-isolation -q

/scratch/edk202/ngram-prep
env: LC_ALL=C.UTF-8
env: LANG=C.UTF-8
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Auto-reload packages
%load_ext autoreload
%autoreload 2

In [2]:
# Standard stuff
from pathlib import Path

# NLTK stuff
from nltk.corpus import stopwords; stopwords = set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer; lemmatizer = WordNetLemmatizer()

# Raw n-gram acquisition stuff
from ngram_acquire.pipeline.orchestrate import download_and_ingest_to_rocksdb
from ngram_acquire.pipeline.logger import setup_logger

# Cython utilities
from ngram_filter.config import PipelineConfig, FilterConfig
from ngram_filter.pipeline.orchestrator import build_processed_db

In [3]:
setup_logger(
    db_path="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

PosixPath('/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams.db/ngram_download_20250928_124057.log')

# **Download 5-Grams and Ingest to a RocksDB Database**

In [4]:
download_and_ingest_to_rocksdb(
    ngram_size = 5,
    repo_release_id = "20200217",
    repo_corpus_id = "eng",
    db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db",
    file_range = (0, 19422),
    random_seed = 11,
    workers = 40,
    use_threads = False,
    ngram_type = "tagged",
    overwrite = False,
    write_batch_size = 100_000,
    open_type = "write:packed24",
    post_compact = True
)

🎉 All files in the specified range are already processed!


## Process the Ngrams

In [ ]:
src_db = Path("/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db")
dst_db = src_db.parent / "5grams_processed.db"
tmp_dir = src_db.parent / "processing_tmp"
wht_path ="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/1grams_processed.db/whitelist.txt"

pipeline_config = PipelineConfig(
    src_db=src_db,
    dst_db=dst_db,
    tmp_dir=tmp_dir,
    readers=2,
    ingestors=16,
    partitioning_sample_rate=0.001,
    prefix_length=6,
    mode="resume",
    force_cache_use=True,
    enable_ingest=True,
    delete_after_ingest=True,
    enable_compact=False,
    progress_every_s=60.0,
)

filter_config = FilterConfig(
    stop_set=stopwords,
    lemma_gen=lemmatizer,
    whitelist_path=wht_path
)

build_processed_db(pipeline_config, filter_config)

N-GRAM FILTER PIPELINE
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Configuration:
════════════════════════════════════════════════════════════════════════════════════════════════════
  Pipeline
  Run mode: resume
  Ingest after filtering: True
  Compact after ingesting: False
  
  Workers
  Num Workers: 2
  Work units: 16
  Profiles: read=read:packed24, write=write:packed24
  Buffer: 100,000 items, 128MB
  
  Files
  Source: /vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db
  Destination: 02/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db
  Input whitelist: .../Google_Books/20200217/eng/5gram_files/1grams_processed.db/whitelist.txt
    All tokens (min count: 1)
  Output whitelist: None
  Loading whitelist...
  Loaded 40,000 tokens

Phase 1: Creating work units...
════════════════════════════════════════════════════════════════════════════════════════════════════
  Resuming existing wo

## Bash One-Liner to Check Work Status
### Run from the `processing_tmp` directory:
```
python3 -c "
import sqlite3
conn = sqlite3.connect('work_tracker.db')
cur = conn.cursor()
results = cur.execute('SELECT status, COUNT(*) FROM work_units GROUP BY status').fetchall()
status_dict = dict(results)
print(f\"Completed: {status_dict.get('completed', 0)}, Processing: {status_dict.get('processing', 0)}, Pending: {status_dict.get('pending', 0)}\")
"
```

In [ ]:
from common_db.api import open_db

db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams.db"

with open_db(db_path, mode="rw") as db:
    db.compact_all()